In [ ]:
!sudo pip3 install keras

In [ ]:
import pandas as pd
import nltk
import re
import string
import keras
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from google.colab import drive
from unicodedata import normalize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder


from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras import layers
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
from unicodedata import normalize

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
drive.mount('/content/drive')

In [ ]:
cols = ["texto", "nota"]
tweet_data = pd.read_csv(
    "/content/drive/MyDrive/Colab Notebooks/Data/alltweets.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="utf8"
)


In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('spanish')

In [ ]:

def clean_data(tweet):
  tweet = BeautifulSoup(tweet,"lxml").get_text()
  #pasamos a minúsculas
  tweet = tweet.lower()
  #Eliminamos las menciones a usuarios
  tweet = re.sub(r"@[a-z0-9]+", ' ', tweet)
  #Eliminamos los enlaces
  tweet = re.sub(r"https?://[a-z0-9./]", ' ', tweet)
  #Nos quedamos con los caracteres importantes
  tweet = re.sub(r"[^a-zÀ-ÿ']", ' ', tweet)
  #Se quitan los espacios en blanco que se hayan generado de más
  tweet = re.sub(r" +", ' ', tweet)   
  #Se eliminan las tildes
  tweet = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", tweet), 0, re.I)       
  #limpiamos ahora con nltk
  notPunctuation = [c for c in tweet if c not in string.punctuation]
  notPunctuation = "".join(notPunctuation)
  words = notPunctuation.split()
  tweet = [word for word in words if word.lower() not in stopwords.words("spanish")]
  return tweet

In [ ]:
tweet_data["texto"].apply(clean_data)

In [ ]:
tweet_train, tweet_test, train_value, test_value = train_test_split(tweet_data["texto"], tweet_data["nota"], test_size=0.2, random_state=38, stratify = tweet_data["nota"])

In [ ]:
print("Entrenamiento entrada: ", tweet_train.shape)
print("Entrenamiento salida: ", train_value.shape)
print("Testing entrada: ", tweet_test.shape)
print("Testing salida: ", test_value.shape)

In [ ]:
vocab_size = 2**16
oov_tok = "<OOV>"
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

In [ ]:
tokenizer.fit_on_texts(tweet_train)
print("Documentos: ", tokenizer.document_count)
print("Palabras: ", tokenizer.num_words)

In [ ]:
train_sequences = tokenizer.texts_to_sequences(tweet_train)
print(train_sequences[0])

In [ ]:
sequence_length = 200
train_padded = pad_sequences(train_sequences, maxlen=sequence_length, padding='post', truncating='post')

In [ ]:
test_sequences = tokenizer.texts_to_sequences(tweet_test)
test_padded = pad_sequences(test_sequences, maxlen=sequence_length, padding='post', truncating='post')

In [ ]:
model = Sequential()

In [ ]:
embedding_dim = 32
model.add(Embedding(vocab_size, embedding_dim, input_length=sequence_length))

In [ ]:
lstm_out = 256
model.add(Bidirectional(LSTM(lstm_out)))

In [ ]:
	
model.add(Dense(10, activation='relu'))

In [ ]:
	
model.add(Dense(1, activation='sigmoid'))

In [ ]:
	
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
checkpoint_filepath = os.getcwd()
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False, monitor='val_loss', mode='min', save_best_only=True)
callbacks = [EarlyStopping(patience=2), model_checkpoint_callback]

In [ ]:
	
history = model.fit(train_padded, train_value, epochs=4, validation_data=(test_padded, test_value), callbacks=callbacks)

In [ ]:
metrics_df = pd.DataFrame(history.history)
print(metrics_df)